# 9장 텐서플로 시작하기

__텐서플로__ 는 수치 계산을 위한 오픈소스 소프트웨어 라이브러리로 대규모 머신러닝에 맞춰 튜닝되어있다.  
먼저 파이썬으로 수행할 계산 그래프를 정의한다음 텐서플로가 최적화된 C++코드를 사용해 이 그래프를 효율적으로 실행시킨다.  
무엇보다도 계산 그래프를 여러 부분으로 나누어 여러 CPU나 GPU에서 병렬로 실행할 수 있다.  
텐서플로는 분산 컴퓨팅도 지원하므로 수백대의 서버에 계산을 나누어 납득할만한 시간안에 대규모 데이터셋으로 거대한 신경망을 훈련시킬 수 있다.

## 9.1 설치

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [1]:
import tensorflow as tf

C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

마지막 줄에서 뭔가 계산이 실행될것 같지만 실제로는 계산이 실행되지않는다. 단지 계산 그래프만 만들어 질 뿐이다. 이 계산 그래프를 평가하려면 텐서플로 세션을 시작하고 변수를 초기화 한 다음 f를 평가해야 한다.

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


위와 같이 계산 그래프를 만들어어서 세션을 시작하고 닫아 주는 과정이 필요하다.  
매번 sess.run()을 하면 번거로운데 다음과 같은 방법을 사용하면 된다.

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [6]:
result

42

with 블록 안에서는 with 문에서 선언한 세션이 기본 세션으로 지정된다.  
x.initializer.run()을 호출 하는것은 tf.get_default_session().run(x.initializer)를 호출하는 것과 같고,  
y.initializer.run()을 호출 하는 것은 tf.get_default_session().run(y.initializer)를 호출하는 것과 같다.

In [7]:
init = tf.global_variables_initializer()#init 노드 준비

with tf.Session() as sess:
    init.run()
    result = f.eval()

각 변수의 초기화를 일일이 실행하는 대신 global_variables_initializer() 함수를 사용할 수 있다.  
이 함수는 초기화를 바로 수행하지 않고 계싼 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성한다.

주피터나 파이썬 셀에서는 Interactivesession을 만드는 편이 편리 할 수 있다.  
일반적인 Sessioin과 다른점은 InteractiveSession이 만들어질때 자동으로 자신을 기본 세션으로 지정한다는 점이다. 그러므로 with 블록을 생성할 필요는 없다(하지만 수동으로 세션 종료해주어야함)

In [8]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


_일반적으로 텐서플로 프로그램은 두 부분으로 나뉜다_   
첫 부분은 계산그래프를 만들고(__구성단계__)  
두 번쨰 부분은 이 그래프를 실행하는 것(__실행단계__)

## 9.3 계산 그래프 관리

노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [9]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

대부분의 경우 이것으로 충분하지만 가끔은 독립적인 계산 그래프를 여러개 만들어야 할 때가 있다 .  
이렇게 하려면 다음과 같이 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 이를 기본 계산 그래프로 사용할 수 있다.

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

## 9,4 노드값의 생애 주기

한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 찾아 먼저 평가한다. 

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


위 코드를 뜯어보면,  
먼저 매우 간단한 그래프를 정의하고 있고, 그 다음 세션을 시작하고 y를 평가 하기위해 계산 그래프를 실행한다.  
텐서플로는 자동으로 y가 x에 의존한다는 것과 x가 w에 읜존한다는 것을 감지하고 있다.  
그래서 먼저 w를 평가하고 그다음에 x를 그다음에 y를 평가해서 y를 반환하는 구조이다.  
중요한점은 이전에 평가된  w와 x를 재사용하지 않는다는 것이다.  
요약하면 위 코드는 w와  x를 두 번 평가한다.

이전 코드처럼 w와 x를 두 번 평가하지 않고 y와 z를 효율적으로 평가하려면 텐서플로가 한 번의 그래프 실행에서 y와 z를 모두 평가하도록 만들어야 한다.  

In [19]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


## 9.5 텐서플로를 이용한 선형 회귀

텐서플로 연산은 여러개의 입력을 받아 출력을 만들 수 있다. 
상수와 변수 연산은 입력이 없고, 입력과 출력은 __텐서__라는 다차원 배열이다.  


In [23]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y) #점곱

with tf.Session() as sess:
    theta_value = theta.eval()
print(theta_value)

[[-3.7225266e+01]
 [ 4.3568176e-01]
 [ 9.3872147e-03]
 [-1.0598953e-01]
 [ 6.3939309e-01]
 [-4.1104349e-06]
 [-3.7780963e-03]
 [-4.2437303e-01]
 [-4.3785891e-01]]
